In [11]:
import pandas as pd
import numpy as np

In [107]:
# fake data 
data = {'orderid':[1,1,1,1,1,2,2,2,3,3,3,3,3,3,3,3],
        'product':["APPLE","BANANA","MILK","BREAD","CHEESE","SKITTLES","BREAD","MILK","MILK","CHEESE","BREAD","TORTILLA CHIPS","GROUND BEEF","VAGASIL","ONIONS","CARROTS"],
        'category':["PRODUCE","PRODUCE","DAIRY","BAKERY","DAIRY","CANDY","BAKERY","DAIRY","DAIRY","DAIRY","BAKERY","SNACKS","DELI","GIRL STUFF","PRODUCE","PRODUCE"],
        'dept':["PRODUCE","PRODUCE","DAIRY","BAKERY","DAIRY","CANDY","BAKERY","DAIRY","DAIRY","DAIRY","BAKERY","SNACKS","DELI","GIRL STUFF","PRODUCE","PRODUCE"],
        'timestamp':["2020-04-10 17:53:45","2020-04-10 18:15:15","2020-04-10 18:45:01","2020-04-10 19:13:13","2020-04-10 19:32:02","2020-04-10 19:41:47","2020-04-10 19:42:59","2020-04-10 20:30:50","2020-04-10 20:37:39","2020-04-10 21:16:53","2020-04-10 21:50:27","2020-04-10 21:58:53","2020-04-10 22:10:47","2020-04-10 22:25:40","2020-04-10 22:26:52","2020-04-10 22:35:08"]
       } 

# Creates pandas DataFrame 
df = pd.DataFrame(data) 
df = df.reset_index(drop=True)

# make sure timestamp is actually a "time"
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %X')

# add a rank column ordered by the timestamp
# pad this with zeros to make sorting easier
df['packorder'] = df.groupby('orderid')['timestamp'].rank(method="dense", ascending=True).astype(int).astype(str).str.zfill(4)

df

,orderid,product,category,dept,timestamp,packorder
0,1,APPLE,PRODUCE,PRODUCE,2020-04-10 17:53:45,0001
1,1,BANANA,PRODUCE,PRODUCE,2020-04-10 18:15:15,0002
2,1,MILK,DAIRY,DAIRY,2020-04-10 18:45:01,0003
3,1,BREAD,BAKERY,BAKERY,2020-04-10 19:13:13,0004
4,1,CHEESE,DAIRY,DAIRY,2020-04-10 19:32:02,0005
5,2,SKITTLES,CANDY,CANDY,2020-04-10 19:41:47,0001
6,2,BREAD,BAKERY,BAKERY,2020-04-10 19:42:59,0002
7,2,MILK,DAIRY,DAIRY,2020-04-10 20:30:50,0003
8,3,MILK,DAIRY,DAIRY,2020-04-10 20:37:39,0001
9,3,CHEESE,DAIRY,DAIRY,2020-04-10 21:16:53,0002


In [108]:
# create a dataframe by orderid with min and max of timestamp
df_time = df.groupby(['orderid']).agg({'timestamp' : [np.min, np.max], 
                                       }).reset_index(drop=False)
df_time.columns = ['orderid','start','end']

# calc time for each order
df_time['fulfillment_time'] = (df_time['end'] - df_time['start']).astype('timedelta64[m]')

# index by orderid for later
df_time = df_time[['orderid','fulfillment_time']].set_index(['orderid'])
df_time

,fulfillment_time
orderid,
1,98.0
2,49.0
3,117.0


In [120]:
pvt_product = df.pivot(index='orderid', columns='packorder',values='product')

# add a prefix to all the column names
pvt_product.columns = ["p" + str(c) for c in pvt_product.columns]

pvt_product.reset_index(drop=False).set_index('orderid')

,p0001,p0002,p0003,p0004,p0005,p0006,p0007,p0008
orderid,,,,,,,,
1,APPLE,BANANA,MILK,BREAD,CHEESE,NaN,NaN,NaN
2,SKITTLES,BREAD,MILK,NaN,NaN,NaN,NaN,NaN
3,MILK,CHEESE,BREAD,TORTILLA CHIPS,GROUND BEEF,VAGASIL,ONIONS,CARROTS


In [121]:
# repeat for other metadata about the products
pvt_department = df.pivot(index='orderid', columns='packorder',values='dept')

# add a prefix to all the column names
pvt_department.columns = ["p" + str(c) + "_dept" for c in pvt_department.columns]

pvt_department.reset_index(drop=False).set_index('orderid')

pvt_department

,p0001_dept,p0002_dept,p0003_dept,p0004_dept,p0005_dept,p0006_dept,p0007_dept,p0008_dept
orderid,,,,,,,,
1,PRODUCE,PRODUCE,DAIRY,BAKERY,DAIRY,NaN,NaN,NaN
2,CANDY,BAKERY,DAIRY,NaN,NaN,NaN,NaN,NaN
3,DAIRY,DAIRY,BAKERY,SNACKS,DELI,GIRL STUFF,PRODUCE,PRODUCE


In [122]:
# join them all together
final = pd.concat([pvt_product,pvt_department,df_time], axis=1, join="inner")
final = final.reindex(sorted(final.columns), axis=1)
final.reset_index(drop=False)

,orderid,fulfillment_time,p0001,p0001_dept,p0002,p0002_dept,p0003,p0003_dept,p0004,p0004_dept,p0005,p0005_dept,p0006,p0006_dept,p0007,p0007_dept,p0008,p0008_dept
0,1,98.0,APPLE,PRODUCE,BANANA,PRODUCE,MILK,DAIRY,BREAD,BAKERY,CHEESE,DAIRY,NaN,NaN,NaN,NaN,NaN,NaN
1,2,49.0,SKITTLES,CANDY,BREAD,BAKERY,MILK,DAIRY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,117.0,MILK,DAIRY,CHEESE,DAIRY,BREAD,BAKERY,TORTILLA CHIPS,SNACKS,GROUND BEEF,DELI,VAGASIL,GIRL STUFF,ONIONS,PRODUCE,CARROTS,PRODUCE
